# GBIF Exploratory Data Analysis 

##  import libraries 

In [1]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

# load city boundary

In [2]:
url = 'https://opendata.vancouver.ca/explore/dataset/facet-grid-boundaries/download/?format=shp&timezone=America/Los_Angeles&lang=en'
city_boundary = gpd.read_file(url)

# load gbif

In [3]:
gbif = pd.read_csv('data/misc/gbif.csv', sep='\t', low_memory=False)

In [4]:
gbif.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,1065573484,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/264412,Plantae,Tracheophyta,Polypodiopsida,Polypodiales,Dryopteridaceae,Polystichum,Polystichum munitum,...,NaN,2013-05-12T03:23:20Z,CC_BY_NC_4_0,vjh,vjh,NaN,NaN,2019-03-19T21:43:51.212Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84
1,1065573958,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/366049,Animalia,Chordata,Amphibia,Anura,Bufonidae,Anaxyrus,Anaxyrus boreas,...,NaN,2013-08-15T12:38:58Z,CC_BY_NC_4_0,James Maughn,James Maughn,NaN,NaN,2019-03-19T22:01:25.296Z,STILLIMAGE,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84
2,1065578258,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/710315,Plantae,Tracheophyta,Polypodiopsida,Polypodiales,Onocleaceae,Onoclea,Onoclea sensibilis,...,NaN,2014-06-01T18:10:11Z,CC_BY_NC_4_0,Killarney Provincial Park,Killarney Provincial Park,NaN,NaN,2019-03-19T23:02:20.517Z,STILLIMAGE,GEODETIC_DATUM_ASSUMED_WGS84
3,1065578318,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/715125,Plantae,Tracheophyta,Polypodiopsida,Osmundales,Osmundaceae,Claytosmunda,Claytosmunda claytoniana,...,NaN,2014-06-03T19:55:29Z,CC_BY_NC_4_0,Killarney Provincial Park,Killarney Provincial Park,NaN,NaN,2019-03-19T23:03:05.983Z,STILLIMAGE,GEODETIC_DATUM_ASSUMED_WGS84
4,1065578594,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/735247,Plantae,Tracheophyta,Polypodiopsida,Osmundales,Osmundaceae,Osmunda,Osmunda regalis,...,NaN,2014-06-13T18:48:27Z,CC_BY_NC_4_0,Killarney Provincial Park,Killarney Provincial Park,NaN,NaN,2019-03-19T23:05:25.989Z,STILLIMAGE,GEODETIC_DATUM_ASSUMED_WGS84


# convert gbif data to geodataframe 

In [5]:
gbif_gdf = gpd.GeoDataFrame(gbif, 
                            crs='epsg:4326', 
                            geometry=gpd.points_from_xy(gbif.decimalLongitude, gbif.decimalLatitude))

# clip gbif data 

In [6]:
vancouver_gbif = gpd.clip(gbif_gdf, mask=city_boundary)

# save clipped species data 

In [14]:
vancouver_gbif.to_file('data/vancouver_gbif.shp')

# load parks 


In [7]:
parks = gpd.read_file('data/vancouver_parks.shp')
parks.head(3)

,LOCAL_REG_,PARK_NAME,PARK_TYPE,PARK_PRIMA,REGIONAL_D,MUNICIPALI,CIVIC_NUMB,CIVIC_NU_1,STREET_NAM,LATITUDE,...,WEBSITE_UR,LICENCE_CO,FEATURE_AR,FEATURE_LE,OBJECTID,SE_ANNO_CA,SHAPE.AREA,SHAPE.LEN,fme_featur,geometry
0,2848,Locarno Park,Local,Park,Metro Vancouver,Vancouver,NaN,None,None,49.274275,...,http://covapp.vancouver.ca/parkfinder/parkdeta...,Contains information licensed under the Open G...,191059.2844,4083.6859,1561313,None,0,0,WHSE_BASEMAPPING.GBA_LOCAL_REG_GREENSPACES_SP,"MULTIPOLYGON (((-123.21241 49.27645, -123.2120..."
1,2849,Macdonald Park,Local,Park,Metro Vancouver,Vancouver,NaN,None,None,49.229759,...,http://covapp.vancouver.ca/parkfinder/parkdeta...,Contains information licensed under the Open G...,10552.0320,423.5468,1561314,None,0,0,WHSE_BASEMAPPING.GBA_LOCAL_REG_GREENSPACES_SP,"POLYGON ((-123.09928 49.23014, -123.09747 49.2..."
2,2850,Maclean Park,Local,Park,Metro Vancouver,Vancouver,NaN,None,None,49.278819,...,http://covapp.vancouver.ca/parkfinder/parkdeta...,Contains information licensed under the Open G...,11898.0851,468.6095,1561315,None,0,0,WHSE_BASEMAPPING.GBA_LOCAL_REG_GREENSPACES_SP,"POLYGON ((-123.08750 49.27845, -123.08757 49.2..."


In [8]:
plot = KeplerGl(height=500)
plot.add_data(data=vancouver_gbif, name='species')
plot.add_data(data=parks, name='parks')
#plot.save_to_html(file_name='van_parks_gbif.html')
plot

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'species': {'index': [81, 82, 83, 84, 85, 86, 199, 247, 435, 436, 437, 504, 1347, 1348, 1349, 2…